# W Brace with Reduced cross-section

This example shows the computation of the factored tension resistance of a W shape used as a tension
member in a lateral brace in a building such as that shown in the following photo.  There are 4 braces
shown.  Note that the upper left brace has the flange tips removed from the W-shape (to ensure 
ductility under seismic forces).  We will compute the strength of a brace similar to that shown
in the photo.

![Cross Braces](images/DSC6443-small.jpg)

Note that this was given as problem PA2, Jan 22, 2015.

Compute the factored tension resistance, $T_r$, of the following assembly.  Steel is G40.21 350W and bolts
are 3/4" ASTM A325 in 22mm punched holes.

![End Details](images/brace1.svg)

Note that 40mm is cut from each flange tip of the W250x67.

In [1]:
from Designer import show, DesignNotes, SST
notes = DesignNotes('Tr',trace=True)

## Tr for Angles

In [2]:
Fy = 350
Fu = 450
d,b,t,Ag = SST.section('L102x76x13','D,B,T,A')
bd = 25.4 * 3/4   # bolt size, mm
ha = 22 + 2  # hole allowance  - punched holes
g1 = 65      # gauge, longer leg
g2 = 45      # gauge, shorter leg
show('d,b,t,Ag,bd,ha,g1,g2')

d  = 102
b  = 76.2
t  = 12.7
Ag = 2100
bd = 19.05
ha = 24
g1 = 65
g2 = 45


![Angle Details](images/angle.svg)

### Net Section Fracture:

In [3]:
# gross width = "flattened" width of angle:
wg = d + b - t

# failure path 1-1: 1 hole
wn1 = wg - 1*ha

# failure path 2-2: 2 holes
g = g1 + g2 - t
s = 80
wn2 = wg - 2*ha + s**2/(4*g)

wn = min(wn1,wn2)
show('wg,g,s,wn1,wn2,wn')

wg  = 165.5
g   = 97.3
s   = 80
wn1 = 141.5
wn2 = 133.9
wn  = 133.9


In [4]:
An = wn*t
Ane = 0.8*An   # S16-14: 12.3.3.2 (b) (i) - connected 1 leg 4 lines of bolts
phiu = 0.75
Tr = 4. * phiu*Ane*Fu * 1E-3     # S16-14: 13.2 a) iii)
notes.record(Tr,'Net section fracture, 4 angles','An,Ane,Tr,Fu');

    Net section fracture, 4 angles: Tr = 1837
       (An=1701, Ane=1361, Fu=450)


### Gross Section Yield:

In [5]:
phi = 0.9
Tr = 4. * phi*Ag*Fy * 1E-3    # S16-14: 13.2 a) i)
notes.record(Tr,'Gross section yield, 4 angles','Ag,Tr');

    Gross section yield, 4 angles: Tr = 2646
       (Ag=2100)


### Block Shear

In [6]:
Agv = (40. + 3*75.)*t
An = (min(d-g1,b-g2) - ha/2.)*t
Ut = 0.3
phiu = 0.75
Tr = 4. * phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
notes.record(Tr,'Block shear, 4 angles','Ut,An,Agv,Tr');

    Block shear, 4 angles: Tr = 2522
       (Ut=0.3, An=243.8, Agv=3366)


## Tr for W Shape

![W Shape](images/w.svg)

In [7]:
Fy = 350.
Fu = 450.
Ag,b,d,t,w = SST.section('W250x67',properties='A,B,D,T,W')
show('Ag,b,d,t,w,Fy,Fu')

Ag = 8550
b  = 204
d  = 257
t  = 15.7
w  = 8.9
Fy = 350
Fu = 450


In [8]:
wp = 190   # width of web reinforcing PL
tp = 8     # thickness of web reinforcing PL
wc = 40    # width cut from flange tips

### Net section fracture

In [9]:
# Path 1-1: net = gross  +  plates  -  holes
An = Ag  +  2*tp*wp   -  2*ha*(w+tp+tp)
Ane = 0.85*An    # S16-14: 12.3.3.2 (c) (i)
phiu = 0.75
Tr = phiu*Ane*Fu * 1E-3
notes.record(Tr,'Net section fracture, W shape','An,Ane,Tr');

    Net section fracture, W shape: Tr = 2982
       (An=10390, Ane=8836)


### Gross section yield

In [10]:
Agr = Ag - 4*wc*t   # reduced area due to flange cuts
phi = 0.9
Tr = phi*Fy*Agr * 1E-3     # S16-14: 13.2 a) i)
notes.record(Tr,'Gross section yield, W shape','Agr,Tr');

    Gross section yield, W shape: Tr = 1902
       (Agr=6038)


### Block Shear

In [11]:
T = w + tp + tp          # thickness of web + reinforcing plates
Agv = 2*(40 + 3*75)*T
An = (g2 + g2 + 25 - ha)*T   # estimate 25mm spacing between angles (gusset thickness)
Ut = 1.0
phiu = 0.75
Tr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3       # S16-14: 13.11
notes.record(Tr,'Block shear, W shape','T,Ut,An,Agv,Tr');

    Block shear, W shape: Tr = 3140
       (T=24.9, Ut=1.0, An=2266, Agv=13200)


### Tearout

In [12]:
Agv = 4*(40 + 3*75)*T
An = 0.
Ut = 1.0
phiu = 0.75
Tr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3      # S16-14: 13.11
notes.record(Tr,'Tearout, W shape','Ut,An,Agv,Tr');

    Tearout, W shape: Tr = 4751
       (Ut=1.0, An=0, Agv=26390)


## Summary

In [13]:
notes.summary()


Summary of DesignNotes for Tr

Values of Tr:
-------------
    Net section fracture, 4 angles: Tr = 1840  <-- governs
    Gross section yield, 4 angles:  Tr = 2650
    Block shear, 4 angles:          Tr = 2520
    Net section fracture, W shape:  Tr = 2980
    Gross section yield, W shape:   Tr = 1900
    Block shear, W shape:           Tr = 3140
    Tearout, W shape:               Tr = 4750

    Governing Value:
    ----------------
       Tr = 1840 


## To do:

- bolts, shear
- bolts, bearing
- weld, reinforcing plates to web